In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re

BUS_PATH="Singapore_bus"
dirs=os.listdir(BUS_PATH)
Bus_FileName_DirList=dirs
Bus_FileName_DirList = sorted(Bus_FileName_DirList)

In [2]:
Bus_Data=pd.read_csv(os.path.join(BUS_PATH,Bus_FileName_DirList[2]),low_memory=False)
# Bus_Data=Bus_Data.iloc[:, 7:]

In [3]:
Bus_Data.head(5)

,Card_Number,Card_Type,Travel_Mode,Bus_Service_Number,Direction,Bus_Trip_Num,Bus_Reg_Num,Boarding_stop_stn,Alighting_stop_stn,Ride_start_date,Ride_start_time,Ride_end_date,Ride_end_time
0,424599,Adult,Bus,184,1.0,2.0,1124.0,43061,43131,2017-01-03,08:32:24,2017-01-03,08:37:21
1,5925417,Senior Citizen,Bus,184,1.0,7.0,1535.0,44229,43099,2017-01-03,16:40:14,2017-01-03,16:44:44
2,2342589,Adult,Bus,184,1.0,6.0,1124.0,12091,42071,2017-01-03,14:56:48,2017-01-03,15:02:58
3,1971082,Senior Citizen,Bus,184,1.0,1.0,1124.0,44229,17179,2017-01-03,05:51:23,2017-01-03,06:21:23
4,4781875,Student,Bus,184,1.0,5.0,1073.0,43099,42109,2017-01-03,12:44:00,2017-01-03,12:53:48


预处理每天的数据

1.清除无效标签项

2.将不满足时间范围的值去除

In [95]:
Bus_Data=Bus_Data[(Bus_Data.Ride_start_time>="05:30:00")&(Bus_Data.Ride_start_time<"23:30:00")]
Bus_Data=Bus_Data[(Bus_Data.Ride_end_time>="05:30:00")&(Bus_Data.Ride_end_time<"23:30:00")]

In [96]:
Bus_start_Data=Bus_Data.sort_values('Ride_start_time')
Bus_end_Data=Bus_Data.sort_values('Ride_end_time')

配置储存数据的矩阵 

T：时间片            66

N：站点数            4831

D：每个站点的数据维度 2

In [3]:
import numpy as np
import torch
import pickle
data_np=np.zeros([59,4850,73,2], dtype = int, order = 'C')

/home/user/Software/miniconda3/envs/pvcgn_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


获取站点对应的数字，以及站点对应数据字典

In [24]:
stn_data=pd.read_csv("./stn_all_data.csv",low_memory=False)
stn_to_num = dict(zip((stn_data['stn_data']),stn_data['stn_data_index']))

生成时间间隔列表

In [25]:
time_data=pd.date_range(start = '05:30:00',periods = 73,freq = '15min')
time_dat=time_data.strftime("%H:%M:%S")
time_dat[0]

'05:30:00'

In [138]:
time_dat[72]

'23:30:00'

In [26]:
for (Bus_FileName,i) in zip(Bus_FileName_DirList,range(0,59)):
    Bus_Data=pd.read_csv(os.path.join(BUS_PATH,Bus_FileName),low_memory=False)
    Bus_Data=Bus_Data.iloc[:, 7:]
    Bus_Data=Bus_Data[(Bus_Data.Ride_start_time>="05:30:00")&(Bus_Data.Ride_start_time<"23:30:00")]
    Bus_start_Data=Bus_Data.sort_values('Ride_start_time')
    print(Bus_FileName)
    for Boarding_stn in Bus_start_Data.Boarding_stop_stn.unique():
        Bus_start_Data_Boarding_stn=Bus_start_Data[Bus_start_Data.Boarding_stop_stn.isin([Boarding_stn])]
        N_num=stn_to_num[Boarding_stn]
        # print(N_num)
        for T_num in range(0,72):
            D_num_0=len(Bus_start_Data_Boarding_stn[(Bus_start_Data_Boarding_stn.Ride_start_time>=time_dat[T_num])&(Bus_start_Data_Boarding_stn.Ride_start_time<time_dat[T_num+1])])
            data_np[i,N_num,T_num,0]=D_num_0



2017-01-01_Bus.csv
2017-01-02_Bus.csv
2017-01-03_Bus.csv
2017-01-04_Bus.csv
2017-01-05_Bus.csv
2017-01-06_Bus.csv
2017-01-07_Bus.csv
2017-01-08_Bus.csv
2017-01-09_Bus.csv
2017-01-10_Bus.csv
2017-01-11_Bus.csv
2017-01-12_Bus.csv
2017-01-13_Bus.csv
2017-01-14_Bus.csv
2017-01-15_Bus.csv
2017-01-16_Bus.csv
2017-01-17_Bus.csv
2017-01-18_Bus.csv
2017-01-19_Bus.csv
2017-01-20_Bus.csv
2017-01-21_Bus.csv
2017-01-22_Bus.csv
2017-01-23_Bus.csv
2017-01-24_Bus.csv
2017-01-25_Bus.csv
2017-01-26_Bus.csv
2017-01-27_Bus.csv
2017-01-28_Bus.csv
2017-01-29_Bus.csv
2017-01-30_Bus.csv
2017-01-31_Bus.csv
2017-02-01_Bus.csv
2017-02-02_Bus.csv
2017-02-03_Bus.csv
2017-02-04_Bus.csv
2017-02-05_Bus.csv
2017-02-06_Bus.csv
2017-02-07_Bus.csv
2017-02-08_Bus.csv
2017-02-09_Bus.csv
2017-02-10_Bus.csv
2017-02-11_Bus.csv
2017-02-12_Bus.csv
2017-02-13_Bus.csv
2017-02-14_Bus.csv
2017-02-15_Bus.csv
2017-02-16_Bus.csv
2017-02-17_Bus.csv
2017-02-18_Bus.csv
2017-02-19_Bus.csv
2017-02-20_Bus.csv
2017-02-21_Bus.csv
2017-02-22_B

In [27]:
file = open('data_np.pickle', 'wb')
pickle.dump(data_np, file)
file.close()

In [104]:
Bus_start_Data.head(5)

,Boarding_stop_stn,Alighting_stop_stn,Ride_start_date,Ride_start_time,Ride_end_date,Ride_end_time
2114902,64381,14141,2017-01-01,05:30:00,2017-01-01,06:35:12
1576786,21379,22009,2017-01-01,05:30:00,2017-01-01,05:33:23
1887647,63269,64031,2017-01-01,05:30:00,2017-01-01,05:42:00
1204703,46239,45139,2017-01-01,05:30:01,2017-01-01,06:11:27
221291,76291,75009,2017-01-01,05:30:02,2017-01-01,05:45:02


In [149]:
Bus_start_Data[Bus_start_Data.Boarding_stop_stn.isin([64381])].head(20)

,Boarding_stop_stn,Alighting_stop_stn,Ride_start_date,Ride_start_time,Ride_end_date,Ride_end_time
2114902,64381,14141,2017-01-01,05:30:00,2017-01-01,06:35:12
2124465,64381,81089,2017-01-01,05:30:02,2017-01-01,05:58:24
1888145,64381,63281,2017-01-01,05:31:11,2017-01-01,05:45:02
2126941,64381,81219,2017-01-01,05:39:36,2017-01-01,06:05:38
2122651,64381,63039,2017-01-01,05:39:38,2017-01-01,05:49:03
176915,64381,63341,2017-01-01,05:43:20,2017-01-01,05:46:56
1875515,64381,63221,2017-01-01,05:44:43,2017-01-01,05:56:21
1788058,64381,84309,2017-01-01,05:45:49,2017-01-01,06:12:34
1785188,64381,84009,2017-01-01,05:45:50,2017-01-01,06:15:35
2131988,64381,70279,2017-01-01,05:52:18,2017-01-01,06:10:50


In [17]:
with open('./data_all_np.pickle', 'rb') as file:
    a_dict1 =pickle.load(file)



In [18]:
a_dict1.shape

(59, 4850, 73, 2)

In [4]:
Reg_List=[]
for Bus_FileName in Bus_FileName_DirList:
    Bus_Data=pd.read_csv(os.path.join(BUS_PATH,Bus_FileName),low_memory=False)
    Reg_List=Reg_List+(list(Bus_Data.Boarding_stop_stn.unique()))
    Reg_List=Reg_List+(list(Bus_Data.Alighting_stop_stn.unique()))
    print(Bus_FileName)
    
a=set(Reg_List)
# a=[]
# for i in Reg_List:
#     a=a+Reg_List
  
stn_list=list(a)
stn_list.sort()
data=pd.DataFrame({'stn_date':stn_list})
data.to_csv("./stn_all_data.csv",index=False)

2017-01-01_Bus.csv
2017-01-02_Bus.csv
2017-01-03_Bus.csv
2017-01-04_Bus.csv
2017-01-05_Bus.csv
2017-01-06_Bus.csv
2017-01-07_Bus.csv
2017-01-08_Bus.csv
2017-01-09_Bus.csv
2017-01-10_Bus.csv
2017-01-11_Bus.csv
2017-01-12_Bus.csv
2017-01-13_Bus.csv
2017-01-14_Bus.csv
2017-01-15_Bus.csv
2017-01-16_Bus.csv
2017-01-17_Bus.csv
2017-01-18_Bus.csv
2017-01-19_Bus.csv
2017-01-20_Bus.csv
2017-01-21_Bus.csv
2017-01-22_Bus.csv
2017-01-23_Bus.csv
2017-01-24_Bus.csv
2017-01-25_Bus.csv
2017-01-26_Bus.csv
2017-01-27_Bus.csv
2017-01-28_Bus.csv
2017-01-29_Bus.csv
2017-01-30_Bus.csv
2017-01-31_Bus.csv
2017-02-01_Bus.csv
2017-02-02_Bus.csv
2017-02-03_Bus.csv
2017-02-04_Bus.csv
2017-02-05_Bus.csv
2017-02-06_Bus.csv
2017-02-07_Bus.csv
2017-02-08_Bus.csv
2017-02-09_Bus.csv
2017-02-10_Bus.csv
2017-02-11_Bus.csv
2017-02-12_Bus.csv
2017-02-13_Bus.csv
2017-02-14_Bus.csv
2017-02-15_Bus.csv
2017-02-16_Bus.csv
2017-02-17_Bus.csv
2017-02-18_Bus.csv
2017-02-19_Bus.csv
2017-02-20_Bus.csv
2017-02-21_Bus.csv
2017-02-22_B

In [9]:
stn_all_data=pd.read_csv('./stn_all_data.csv')


In [13]:
stn_all_data.insert(1,"stn_data_index",range(0,len(stn_all_data.stn_data)))


In [15]:
stn_all_data.to_csv("./stn_all_data.csv",index=False)

In [19]:
with open('./data_all_np.pickle', 'rb') as file:
    a_dict1 =pickle.load(file)



In [20]:
a_dict1[0,0:20,:,:]

array([[[ 0,  0],
        [ 0,  0],
        [ 0,  0],
        ...,
        [ 0,  0],
        [ 0,  0],
        [ 0,  0]],

       [[ 4,  0],
        [ 7,  2],
        [11,  0],
        ...,
        [43,  3],
        [13,  1],
        [ 0,  0]],

       [[ 0,  0],
        [15,  0],
        [11,  0],
        ...,
        [20,  2],
        [22,  8],
        [ 0,  0]],

       ...,

       [[ 0,  0],
        [ 2,  2],
        [ 3,  6],
        ...,
        [ 9,  4],
        [ 4,  3],
        [ 0,  0]],

       [[ 0,  0],
        [ 0,  0],
        [ 0,  0],
        ...,
        [ 4,  0],
        [ 4,  0],
        [ 0,  0]],

       [[ 0,  0],
        [ 0,  0],
        [ 2,  0],
        ...,
        [ 2,  2],
        [ 0, 10],
        [ 0,  0]]])

## 获得最多的站点

In [ ]:
max_stn=0
for (Bus_FileName,i) in zip(Bus_FileName_DirList,range(0,59)):
    Bus_Data=pd.read_csv(os.path.join(BUS_PATH,Bus_FileName),low_memory=False)
    unique_stations = len(Bus_Data.Boarding_stop_stn.unique())
    if unique_stations > max_stn:
        max_stn = unique_stations
        print(f"File {Bus_FileName} with most unique stations: {unique_stations} ")

2017-01-01_Bus.csv
2017-01-03_Bus.csv
2017-01-04_Bus.csv
2017-01-09_Bus.csv
2017-01-13_Bus.csv
2017-01-25_Bus.csv
2017-02-15_Bus.csv
2017-02-21_Bus.csv


In [ ]:
Bus_Data=pd.read_csv(os.path.join(BUS_PATH,"2017-02-21_Bus.csv"),low_memory=False)

Bus_Data=Bus_Data[(Bus_Data.Ride_start_time>="05:30:00")&(Bus_Data.Ride_start_time<"23:30:00")]
Bus_Data

In [ ]:
import os
import re

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd


BUS_PATH="../DATA/Singapore_bus"
dirs=os.listdir(BUS_PATH)
Bus_FileName_DirList=dirs


Bus_Data=pd.read_csv(os.path.join(BUS_PATH,"2017-02-21_Bus.csv"),low_memory=False)

Bus_Data=Bus_Data[(Bus_Data.Ride_start_time>="05:30:00")&(Bus_Data.Ride_start_time<"23:30:00")]

def df_insert(df, n, arr, ignore_index=True):
    df1 = df.iloc[:n]
    df2 = df.iloc[n:]
    df0 = df1.append([arr], ignore_index).append(df2, ignore_index)
    return df0

Bus_Data['a']=' '
Bus_Data.Ride_start_time=pd.to_datetime(Bus_Data.Ride_start_date+Bus_Data.a+Bus_Data.Ride_start_time)
Bus_Data.Ride_end_time=pd.to_datetime(Bus_Data.Ride_end_date+Bus_Data.a+Bus_Data.Ride_end_time)
temp_stn=[]
all_max_stn=[]
all_max_stn_bus_num=[]
for Bus_Service_Number_temp in Bus_Data.Bus_Service_Number.unique():
# for Bus_Service_Number_temp in ['125']:
    
    Bus_Data_xxx=Bus_Data.loc[(Bus_Data['Direction']==0.0)&(Bus_Data['Bus_Service_Number']==Bus_Service_Number_temp)]
    for Bus_Reg_Num in Bus_Data_xxx.Bus_Reg_Num.unique():
        for Bus_Trip_Num in Bus_Data_xxx.loc[(Bus_Data_xxx['Bus_Reg_Num']==Bus_Reg_Num)].Bus_Trip_Num.unique():
            bus_xxx=Bus_Data_xxx.loc[(Bus_Data_xxx['Bus_Reg_Num']==Bus_Reg_Num)&(Bus_Data_xxx['Bus_Trip_Num']==Bus_Trip_Num)]

            new_bus_stn_bo=pd.DataFrame({'Stop_stn':bus_xxx.Boarding_stop_stn,'Ride_time':bus_xxx.Ride_start_time})
            new_bus_stn_al=pd.DataFrame({'Stop_stn':bus_xxx.Alighting_stop_stn,'Ride_time':bus_xxx.Ride_end_time})

            new_add_bo_al=new_bus_stn_bo.merge(new_bus_stn_al,how='outer').sort_values('Ride_time')
            new_add_bo_al

            a=new_add_bo_al
            # a['Ride_time']=pd.to_datetime(a.Ride_time)
            a.drop_duplicates('Stop_stn','first',inplace=True)

            a['sub']=pd.to_timedelta(0)

            for i in range(1,a.shape[0]):
                if  a.iloc[i,1]-a.iloc[i-1,1]<pd.to_timedelta("15min"):
                    a.iloc[i,2]=a.iloc[i,1]-a.iloc[i-1,1]+a.iloc[i-1,2]
                else:
                    a=a.iloc[:i,:]
                    break
                
            a=a.reset_index(drop=True)
            if a.shape[0]>6:
                temp_stn.append(a)
    if len(temp_stn)>0:
        pass
    else:
        continue
    def takeSecond(elem):
        return elem.shape[0]
    temp_stn.sort(key=takeSecond,reverse=True)

    

    temp_max_stn=temp_stn[0]
    test_stn=[]
    for i in range(1,len(temp_stn)-3):
        t=pd.concat([temp_stn[i], temp_max_stn, temp_max_stn]).Stop_stn.drop_duplicates(keep=False)
        t_len=len(t)
        if t_len>0:
            if t.index[0]==0:

                continue
            else:
                for t_index in t.index:
                    t_stn=temp_stn[i].at[t_index-1,'Stop_stn']
                    t_last_index=temp_max_stn[temp_max_stn.Stop_stn==t_stn].index.tolist()
                    t_next_index=t_last_index[0]+1
                    max_sub_time=temp_max_stn.loc[t_next_index:,'sub']-temp_max_stn.loc[t_last_index[0],'sub']
                    temp_sub_time=temp_stn[i].at[t_index,'sub']-temp_stn[i].at[t_index-1,'sub']
                    num_index=0
                    for sub_time in max_sub_time:
                        if temp_sub_time>sub_time:
                            num_index+=1
                        else:
                            break

                    temp_max_stn=df_insert(temp_max_stn,t_last_index[0]+num_index+1,temp_stn[i][temp_stn[i].Stop_stn.isin([temp_stn[i].at[t_index,'Stop_stn']])])

    print(Bus_Service_Number_temp)
    all_max_stn_bus_num.append(str(Bus_Service_Number_temp))
    all_max_stn.append(temp_max_stn)


In [24]:

bus_dict = dict(zip(all_max_stn_bus_num, all_max_stn))

In [27]:
import pickle
file = open('phy_graph.pickle', 'wb')
pickle.dump(bus_dict, file)
file.close()

In [4]:
import pickle
with open('./phy_graph_0_new.pickle', 'rb') as file:
    phy_graph_0 =pickle.load(file)
with open('./phy_graph_1_new.pickle', 'rb') as file:
    phy_graph_1 =pickle.load(file)

In [5]:
server_del_key = ['184', '185', '105', '961', '854', '162', '20', '142', '950', '56', '804', '57', '139', '179', '969', '96', '190', '70', '225', '904', '192', '183', '107', '229', '75', '960', '979M', '123', '913', '18', '140', '262', '55', '506', '92', '72', '7', '129', '118', '251', '132', '852', '859', '66', '166', '410', '26', '136', '81', '963', '856', '109', '325', '240', '86', '302', '33', '155', '63M', '922', '22', '59', '100', '181', '17B', '803', '11', '85', '133', '243', '812', '255', '82', '54', '48', '170', '990', '292', '31', '985', '358', '197', '122', '8', '37', '159', '174', '857', '74', '196', '98M', '151', '111', '45', '300', '63', '189', '172', '222', '911', '139M', '167', '165', '920', '52', '171', '89', '4', '119', '13', '163', '965', '77', '200', '187', '156', '983', '115', '15', '78', '30', '50', '700', '382', '855', '79', '193', '941', '19', '62', '87', '141', '882', '36', '307', '853', '147', '900', '93', '130', '124', '252', '97', '188', '83', '927', '161', '359', '241', '980', '945', '400', '972', '315', '32', '198', '23', '145', '163M', '175', '60', '354', '182', '947', '3', '143', '46', '134', '256', '293', '38', '160', '925', '770', '112', '170X', '27', '67', '14', '806', '106', '61', '64', '186', '970', '101', '157', '979', '16', '176', '138', '51', '53', '851', '28', '90', '860', '5', '127', '249', '88', '371', '42', '65', '84', '201', '386', '116', '238', '6', '153', '94', '9', '131', '253', '10', '168', '858', '98', '120', '242', '35', '41', '301', '231', '43', '24', '903', '17', '268', '912', '135', '257', '43M', '811', '12', '966', '246', '975', '39', '333', '76', '169', '113', '334', '235', '150', '272', '335', '883', '102', '308', '901', '261', '49', '180', '103', '2', '91', '80', '53M', '123M', '372', '800', '403', '324', '117', '964', '154', '282', '69', '859A', '191', '228', '21', '265', '58', '25', '95', '254', '47', '285', '291', '178', '121', '518', '158', '99', '73', '402', '182M', '195', '317', '232', '962', '34', '173', '825', '269', '258', '502', '125', '40', '199', '143M', '29', '273', '805']

In [6]:
line_station = []
for i in server_del_key:
    line_station.append(phy_graph_1[i].Stop_stn.to_list())

In [7]:
flattened_set = set(item for sublist in line_station for item in sublist)
unique_flattened_list = list(flattened_set)

In [8]:
len(unique_flattened_list)

4401

In [9]:
unique_flattened_list

[98309,
 65541,
 98311,
 24581,
 49159,
 11,
 98319,
 65551,
 49169,
 24599,
 24601,
 49179,
 24609,
 24611,
 49189,
 24619,
 24621,
 49199,
 41011,
 24629,
 24631,
 41019,
 41021,
 24639,
 24641,
 41029,
 41031,
 24651,
 41041,
 41049,
 82009,
 41051,
 82011,
 24671,
 41061,
 24679,
 24681,
 41069,
 82029,
 41071,
 82032,
 24689,
 24691,
 41079,
 41081,
 24699,
 41089,
 82049,
 82051,
 82059,
 82061,
 41101,
 41109,
 82069,
 82071,
 41111,
 41119,
 82079,
 82081,
 41121,
 41129,
 82089,
 82091,
 41131,
 41139,
 82099,
 41141,
 82109,
 41149,
 41151,
 41159,
 82119,
 82129,
 82139,
 82149,
 82159,
 33011,
 33019,
 33021,
 82179,
 33029,
 33031,
 65801,
 33039,
 33041,
 33049,
 74011,
 33051,
 33059,
 74021,
 82221,
 74031,
 82231,
 74041,
 25009,
 66009,
 66011,
 66019,
 25061,
 66021,
 66029,
 66031,
 25071,
 66039,
 66041,
 16891,
 66049,
 66051,
 25099,
 66059,
 66061,
 25109,
 66069,
 66071,
 25119,
 66079,
 66081,
 25129,
 66089,
 66091,
 66099,
 66101,
 66109,
 66111,
 66119,
 66

In [10]:
def diff(listA,listB):
    #求交集的两种方式
    retA = [i for i in listA if i in listB]
    retB = list(set(listA).intersection(set(listB)))
    
    print ("retA is: ",retA)
    print ("retB is: ",retB)
    
    #求并集
    retC = list(set(listA).union(set(listB)))
    print ("retC1 is: ",retC)
    
    #求差集，在B中但不在A中
    retD = list(set(listB).difference(set(listA)))
    print ("retD is: ",retD)
    
    retE = [i for i in listB if i not in listA]
    print ("retE is: ",retE)

In [11]:
import pandas as pd
import numpy as np

data=np.load("./temp_mat_v1_new.npy")
data=data+np.eye(4401)

In [2]:
# 单位矩阵
np.eye(4401)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [12]:
data

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [13]:
origin_id=[([i]*len(data[:,0]))for i in range(len(data[:,0]))]
destination_id=[[j for j in range(0,len(data[:,0]))]for i in range(len(data[:,0]))]
connection=[list(data[i,:])for i in range(len(data[:,0]))]

In [14]:
origin_id=[i for item in origin_id for i in item]
destination_id=[i for item in destination_id for i in item]
connection=[i for item in connection for i in item]

In [15]:
rel_id=[i for i in range(len(origin_id))]
type=['geo']*len(origin_id)

In [17]:
d={'rel_id':rel_id,'type':type,'origin_id':origin_id,'destination_id':destination_id,'connection':connection}
data=pd.DataFrame(data=d)

In [18]:
data.to_csv('./URBAN_BUS.rel',index=False)

In [19]:
ab = [[1,2,3], [5,8], [7,8,9]]
print([i for item in ab for i in item])


[1, 2, 3, 5, 8, 7, 8, 9]


In [20]:
import pandas as pd
import numpy as np

data=np.load("./temp_mat_v1_new.npy")

In [21]:
data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
sum(map(sum,data))/2


6950.5